In [48]:
import os
import json

key_frame_folder = '/Users/huyenphung/Desktop/backend/keyframe_infomaton/keyframe'

def get_image_paths(single_key_frame_folder, video_ID):
    image_video_dict = {}
    idx = 0
    for img_name in sorted(os.listdir(single_key_frame_folder)):
        img_path = os.path.join(single_key_frame_folder, img_name)
        frame_timestamp = img_name.replace('.jpg','')
        image_video_dict[idx] = {'frame_path': img_path, 'video_ID': video_ID, 'timestamp': frame_timestamp}
        idx += 1
    return image_video_dict

def save_json(data, output_folder, filename):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Construct the full file path
    file_path = os.path.join(output_folder, filename)
    
    # Save the data as a pretty-printed JSON file
    with open(file_path, 'w') as outfile:
        json.dump(data, outfile, indent=4)  # Using indent=4 for pretty printing

# Loop through all keyframe folders and save a JSON file for each video
def process_keyframe_folders(multiple_key_frame_folder, output_folder):
    for video in sorted(os.listdir(multiple_key_frame_folder)):
        if '_reduced' in video:
            continue
        video_ID = ''
        single_key_frame_folder = os.path.join(multiple_key_frame_folder, video)
        
        # Get image info for this video
        image_info_dict = get_image_paths(single_key_frame_folder, video_ID)
        
        # Save the dictionary as a JSON file with video name
        filename = f'{video}.json'
        save_json(image_info_dict, output_folder, filename)

# Run the process
output_folder = '/Users/huyenphung/Desktop/backend/keyframe_infomaton/annotation'
process_keyframe_folders(key_frame_folder, output_folder)
